In [10]:
import time
import datetime
import requests
from selenium import webdriver
from bs4 import BeautifulSoup

In [22]:
start_time = time.time()
CHEMBL = "https://www.ebi.ac.uk/chembl/beta/compound_report_card/CHEMBL"
driver = webdriver.Chrome()

SMILESdata = []
counter = 0
for i in range(10):
    CHEMBL+=str(i+1)

    driver.get(CHEMBL)
    time.sleep(.25)
    content = driver.page_source.encode('utf-8').strip()
        
    CHEMBLsoup = BeautifulSoup(content, "html.parser")
    table = CHEMBLsoup.find_all("table", class_="compact-table hide-on-small-only")[0]
    table_body = table.find("tbody")
    row = table_body.find_all('tr')[1]
    col = row.find("td", class_="BCK-CanonicalSmiles")
    SMILES = str(col)[140:].split(">",1)[0]
    length = len(str(col)[140:].split(">",1)[0])
    SMILES = SMILES[:length-1]
    if ((len(SMILES))!=0):
        SMILESdata.append(SMILES)
        counter+=1
    elapsed = time.time() - start_time
    hours = int(elapsed/3600)
    minutes = int(int(elapsed/60)%60)
    seconds = int(elapsed%60)
    print(" SMILES saved:", counter, " | Time elapsed:", hours, "hours", minutes, "minutes", seconds, "seconds" )
    
    CHEMBL = "https://www.ebi.ac.uk/chembl/beta/compound_report_card/CHEMBL"

driver.close()
print(*SMILESdata, sep="\n")

 SMILES saved: 1  | Time elapsed: 0 hours 0 minutes 18 seconds
COc1ccc2[C@@H]3[C@H](COc2c1)C(C)(C)OC4=C3C(=O)C(=O)C5=C4OC(C)(C)[C@@H]6COc7cc(OC)ccc7[C@H]56

